In [1]:
!pip3 install -q --upgrade langchain langchain-google-genai google-generativeai
!pip3 install -q python-dotenv

## Aula 02 — RAG

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

In [ ]:
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")